# <font color=green> Machine Learning: Validação de modelos
---

# ---------------------------------------VALIDAÇÃO CRUZADA E ALEATORIEDADE---------------------------------------

    * Utilizar o método read_csv do pandas;
    * O que é a validação cruzada(cross validation);
    * Utilizar o std para saber o desvio padrão;
    * Árvore de classificação.

In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


* Recuperando dados de treino e teste:

In [2]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


* Dummy

In [3]:
dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi %.2f%%" % acuracia)    

A acurácia do dummy stratified foi 58.00%


* DecisionTree

In [4]:
SEED = 158020
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


# ALTERANDO O SEED:

* AO ANALISAR OS RESULTADOS COM SEEDs DIFERENTES, TEMOS RESULTADOS DIFRENTES, PORÉM A NOSSA DECISÃO NÃO PODE DEPEDER DE UMA ALEATORIEDADE! ENTÃO DEVEMOS MINIMIZAR O EFEITO DESSA ALEATORIEDADE DO TREINO E TESTE NA NOSSA DECISÃO. E A SOLUÇÃO PARA ISSO É A VALIDAÇÃO CRUZADA

# CROSS VALIDATE:

* PARA TRABALHARMOS COM A VALIDAÇÃO CRUZADA
* AGORA INDEPENDENTE DO VALOR DO MEU SEED, MEUS RESULTADOS SERÃO OS MESMOS
* Obs.: 95% da densidade estará dentro da média, mais ou menos dois desvios padrões. Então, desvio padrão é o quanto estamos desviando dessa média
* Obs.: CV = qts partes queremos quebrar nossos dados
* Obs.: return_train_score=False para retornarmos apenas os dados de teste, e não os de treino!

In [5]:
SEED = 5
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 10 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 10 = [74.24, 77.32]


In [7]:
SEED = 515
np.random.seed(SEED)

modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = 10, return_train_score=False)
media = results['test_score'].mean()
desvio_padrao = results['test_score'].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

Accuracy com cross validation, 5 = [74.24, 77.32]


# ---------------------------------------k-FOLD E ALEATORIEDADE---------------------------------------

    * Utilizar o método KFold para embaralhar(shuffle).

* O Cross Validate não recebe o parâmetro de aleatoriedade, e esse é o padrão. Nós sabemos em quantos pedaços ele quebrará os dados, porque somos nós quem decidimos essa quantidade
* Por mais que recebamos todos os dados em uma sequência, não queremos quebrar eles em 5 pedaços e, a partir disso, fazer o cross_validate. O ideal seria embaralhar estes dados e, então executar a validação cruzada.

In [8]:
from sklearn.model_selection import KFold

In [9]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [10]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

In [11]:
def imprime_resultado(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio %.2f" % (media * 100))
    print("Accuracy intervalo = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

In [12]:
cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultado(results)

Accuracy médio 75.78
Accuracy intervalo = [74.37, 77.19]


* Shuffle = True para embaralhar meus dados antes de fazer a validação cruzada (quebrar em N pedaços)

In [13]:
cvv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cvv, return_train_score=False)
imprime_resultado(results)

Accuracy médio 75.78
Accuracy intervalo = [73.56, 78.00]


# ---------------------------------------ESTRATIFICAÇÃO---------------------------------------

    * Simular uma situação com dados;
    * Ordenar nosso dataframe utilizando o sort_value(“vendido“’);

* Estudamos uma questão de sorte e azar em que a validação era simples: treino-teste. Tínhamos o azar de que nossos dados, no momento de realmente utilizá-los, eram separados como, por exemplo, vendido ou não vendido.
* Como resultado, no treino estavam quase todos que não foram vendidos e no teste quase todos que foram vendidos. Ou seja, ocorria um desbalanceamento das classes. Então, se tivermos azar, o que pode acontecer?
* Com o train_test_split, tínhamos o Stratify como parâmetro, que definia a proporção. Por exemplo, se temos 70% dos dados na classe 0 (não vendido) e 30% na classe 1 (vendido), ele distribui 70% e 30% tanto para treino, quanto para teste. No entanto, esse parâmetro não existe no cross_validate do KFold, que possui apenas os parâmetros n_splits, shuffle e random_state. Sendo assim, não está estratificando.

In [14]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [15]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)

In [16]:
def imprime_resultado(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio %.2f" % (media * 100))
    print("Accuracy intervalo = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

#### ----> SIMULAR SITUAÇÃO HORRÍVEL, DE AZAR:

In [17]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano
4999,74023.29,0,12,24812.80412
5322,84843.49,0,13,23095.63834
5319,83100.27,0,19,36240.72746
5316,87932.13,0,16,32249.56426
5315,77937.01,0,15,28414.50704


* Sem shuffle:

In [18]:
cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cv, return_train_score=False)
imprime_resultado(results)

Accuracy médio 57.84
Accuracy intervalo = [34.29, 81.39]


* Com shuffle:
    * Percebe-se que com o embaralhamento, nossas situações de azar são minimizadas!

In [19]:
cvv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cvv, return_train_score=False)
imprime_resultado(results)

Accuracy médio 75.78
Accuracy intervalo = [73.43, 78.13]


---------------------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------------------

#### ----> SIMULAR SITUAÇÃO NORMAL:

In [20]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

* Sem shuffle:

In [21]:
cv = KFold(n_splits = 10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cv, return_train_score=False)
imprime_resultado(results)

Accuracy médio 75.78
Accuracy intervalo = [74.37, 77.19]


* Shuffle = True para embaralhar meus dados antes de fazer a validação cruzada (quebrar em N pedaços)

In [22]:
cvv = KFold(n_splits = 10, shuffle = True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cvv, return_train_score=False)
imprime_resultado(results)

Accuracy médio 75.78
Accuracy intervalo = [72.21, 79.35]


---------------------------------------------------------------------------------------------------------------

-----------------------------------------------------------------------------------------------------------

#### ----> Testando outros separadores

* Além do Kfold, também temos o separador "StratifiedKFold", que além de separar meus dados para a validação cruzada, também estratifica nossos dados para manter a proporção das minhas classes tanto no treino como no teste

In [23]:
cvv = StratifiedKFold(n_splits = 10, shuffle = True, )
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x, y, cv = cvv, return_train_score=False)
imprime_resultado(results)

Accuracy médio 75.78
Accuracy intervalo = [73.27, 78.29]


# ---------------------------------------DADOS AGRUPÁVEIS---------------------------------------

    * Gerar dados aleatórios com o randint;
    * Gerar uma nova coluna de dados;
    * Utilizar o comando unique para mostrar os modelos únicos;
    * Utilizar o GroupKFold para agrupar por grupo.

* É preciso tomar cuidado quando formos separar o treino e o teste. Temos que pensar que, ao treinarmos nosso algorítimo, nós treinamos ele com os dados; quando nós testamos, temos que ser capazes de testar como se fosse no mundo real.
* No mundo real em um hospital, fulano é um paciente novo, nunca visto, e aquilo que fora treinado com outros pacientes precisa funcionar com ele também. Na hora de separarmos treino e teste, teríamos de agrupar pelos pacientes. Nós não poderíamos colocar dados da Ana no treino e no teste, porque cada paciente é uma pessoa nova, no mundo real do hospital. 
* E se nos treinos e testes temos repetições de pacientes, é possível que o algorítimo não generalize bem, para novos pacientes.

In [24]:
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GroupKFold

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
import pandas as pd

In [25]:
def imprime_resultado(results):
    media = results['test_score'].mean()
    desvio_padrao = results['test_score'].std()
    print("Accuracy médio %.2f" % (media * 100))
    print("Accuracy intervalo = [%.2f, %.2f]" % ((media - 2 * desvio_padrao)*100, (media + 2 * desvio_padrao) * 100))

In [26]:
uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


### Gerando tabela modelo_carro no nosso dataFrame sem valores negativos. Essa tabela vai ser usada para separarmos os grupos e agrupa-los

In [27]:
# len(dados) = 10000
#np.random.randint(-2, 3, size = 1000) => gera 10.000 nmrs entre -2 e 2
dados['modelo'] = dados.idade_do_modelo + np.random.randint(-2, 3, size=10000)

# dados.modelo_aleatorio.unique() => Vou perceber que tenho um modelo = -1
dados.modelo = dados.modelo + abs(dados.modelo.min())

In [28]:
dados.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
0,30941.02,1,18,35085.22134,19
1,40557.96,1,20,12622.05362,19
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,6


In [29]:
dados.modelo.value_counts()

19    925
17    799
18    780
16    689
15    681
20    673
14    635
13    579
21    548
12    531
11    488
10    425
22    412
9     400
8     315
7     271
6     222
23    203
5     180
4     119
3      69
2      28
1      23
0       5
Name: modelo, dtype: int64

## GroupKFold:

* Capaz de separar treino e teste na validação cruzada, de acordo com os grupos.

In [30]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

,preco,vendido,idade_do_modelo,km_por_ano,modelo
4999,74023.29,0,12,24812.80412,15
5322,84843.49,0,13,23095.63834,13
5319,83100.27,0,19,36240.72746,18
5316,87932.13,0,16,32249.56426,19
5315,77937.01,0,15,28414.50704,14


In [31]:
cvv = GroupKFold(n_splits = 10) #GroupKFold
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo, x_azar, y_azar, cv = cvv, groups = dados.modelo, return_train_score=False) # Passo aqui meu groups com o nosso modelo para o agrupamento
imprime_resultado(results)

Accuracy médio 75.78
Accuracy intervalo = [73.48, 78.09]


# ---------------------------------------PIPELINE DE TREINO E VALIDAÇÃO---------------------------------------

    * CV com StandardScaler (reesc)
    * Treinar usando o módulo SVC
    * O que é pipeline
    * Utilizar o pipeline

## CV com StandardScaler:

In [34]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 74.40%


In [36]:
# Aqui faço o fit somente uma vez, porém, deve ser feito em todos os grupos do KFold, por isso usamos o pipeline ali em baixo
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)


from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits = 10)
modelo = SVC()
results = cross_validate(modelo, x_azar_escalado, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultado(results)

Accuracy médio 76.68
Accuracy intervalo = [74.82, 78.54]


## PIPELINE

 * Toda vez que rodamos o GroupKFold, temos que fazer o fit() com uma parte de treino e rodar a validação com o teste. Se tivermos outro treino, rodaremos fit() para esse outro treino, e o teste com o resto. Assim, teremos um processo de duas fases: ESCALAR E VALIDAR
 * Esse processo deve ser rodado várias vezes, de acordo com os nossos grupos e com nossos com nossos splits. Então, precisamos pensar em outra forma, além de fazer o fit() somente uma vez e, depois, rodar o GroupKFold.
 * Ai que vem o Pipeline: primeiro uma transformação, que é o Standrscaler(), e depois um estimador que é um SVC().
 * E vamos rodar esse código. Agora sim, para cada um dos processos de fold, serão rodados tanto a transformacao, quanto o estimador. Depois, outro conjunto de dados será separado em dois, tanto a transformacao, quanto o estimador. Portanto, a execução do código vai demorar um pouco mais. O importante é que teremos uma estimativa mais realista do modelo SVC(), com StandardScaler, no mundo real, considerando agrupamentos. 

In [38]:
from sklearn.pipeline import Pipeline

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([('transformacao',scaler), ('estimador',modelo)])

cv = GroupKFold(n_splits = 10)
results = cross_validate(pipeline, x_azar, y_azar, cv = cv, groups = dados.modelo, return_train_score=False)
imprime_resultado(results)

Accuracy médio 76.70
Accuracy intervalo = [74.84, 78.56]


* Reparem que o valor anterior e este não são tão distantes, mas é preciso entender que a forma feita anteriormente estava errada, porque rodávamos o scaler somente uma vez, para depois rodar os KFolds. Não é questão de estar melhor ou pior, pois estamos falando de um processo, que deve ser rodado para cada uma das fases de treino. Portanto, se tivermos 10 fases de treino, teremos que rodar o scaler para cada uma do conjunto de treino. E é isso que o Pipeline faz para nós, de forma muito mais prática. Basta inserir todas as fases nele e rodar. Claro, pode demorar um pouco mais, mas é a opção que faz mais sentido.